# 计算文件各种类型、属性的数量
### 测试有多少个值未知

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import math
from collections import defaultdict
from functools import reduce

In [ ]:
def validate(filename, headers):
    count_class = {class_name:0 for class_name in list(filter(lambda x : x[0] == "class", headers))[0][1]}
    count_nominal = {att:defaultdict(int) for att,att_type in headers if att != "class" and att_type < 2}
    count_numeric = defaultdict(int)
    count_numeric_unknown = defaultdict(int)
    with open(filename,"r") as file:
        for line in file:
            
            cels = line.strip().split(",")
            if not cels or not cels[0] :
                continue               
            assert len(cels) == len(headers)
            
            i = 0
            for name,data_type in headers:
                val = cels[i]
                if name == "class":
                    assert val in data_type
                    count_class[val] += 1
                elif data_type < 2:
                    count_nominal[name][val] += 1
                else:
                    assert val.isdigit() or val == "?" 
                    if val.isdigit():
                        count_numeric[name] += 1
                    else:
                        count_numeric_unknown[name] += 1 
                i+=1
    
    total = reduce(lambda x, y: x+y, count_class.values())
    print("total instances: ", total)
    print("{0}:".format("class"), " / ".join(["{0} {1}".format(key, val) for key,val in sorted(count_class.items())]))
    print("--------------------attributes--------------------")
    for name,data_type in headers:
        if name != "class":
            if data_type < 2:
                assert total == reduce(lambda x, y: x+y, count_nominal[name].values())
                print("{0}:\n   ".format(name), "\n    ".join(["{0} {1}".format(key, val) for key,val in sorted(count_nominal[name].items())]))
            else:
                assert total == count_numeric[name] + count_numeric_unknown[name]
                print("{0}: known {1} / unkown {2}".format(name, count_numeric[name], count_numeric_unknown[name]))
            print()


In [ ]:
#headers顺序按文件顺序， 是第几个就放第几个
#class也按顺序,第一列就摆第一个
#0=nominal, 1=ordinal, 2=numeric

headers = (
    ("age", 2),
    ("workclass", 0),
    ("fnlwgt", 2),
    ("education", 1),
    ("education-num", 1),
    ("marital-status", 0),
    ("occupation", 0),
    ("relationship", 0),
    ("race", 0),
    ("sex", 0),
    ("capital-gain", 2),
    ("capital-loss", 2),
    ("hours-per-week", 2),
    ("native-country", 0),
    ("class", ("<=50K" , ">50K"))
)

validate("datasets/adult.data", headers)

In [24]:
class ClassCounter:
    def __init__(self, subject):
        self.subject = subject

    def prepare(self, class_num):
        self.__classes_count = [0] * class_num
        self.__classes_prob = [0] * class_num
    
    def add(self, class_index):
        self.__classes_count[class_index] += 1
    
    def run(self):
        total = sum(self.__classes_count)
        for index, val in enumerate(self.__classes_count):
            self.__classes_prob[index] = val/total
    
    def get(self):
        return tuple(self.__classes_prob)
    
class AttrCounter:
    def __init__(self, name):
        self.name = name

    def prepare(self, class_num):
        return
    
    def add(self, attr, class_index):
        return 
    
    def run(self):
        return 
    
    def get(self, attr):
        return 0
    
class NomCounter(AttrCounter):
    __smooth_alpha = 1
    def __init__(self, name):
        super().__init__(name)

    def prepare(self, class_num):
        self.__attr_vals = set()
        self.__attr_count = [defaultdict(int) for i in range(class_num)]
        self.__attr_prob = [defaultdict(float) for i in range(class_num)]      
    
    def add(self, attr, class_index):
        self.__attr_vals.add(attr)
        self.__attr_count[class_index][attr] += 1
    
    def run(self):
        dimension = len(self.__attr_vals)
        for index, attrs in enumerate(self.__attr_count):
            total = sum(attrs.values())
            for val in self.__attr_vals:
                self.__attr_prob[index][val] = self._laplace_smooth(attrs[val], total, dimension)
#         print(self.__attr_vals)
#         print(self.__attr_count)
#         print(self.__attr_prob)
#         print("____________")
    
    def get(self, attr): 
        return tuple(prob[attr] for prob in self.__attr_prob)

    def _laplace_smooth(self, num, total, dimension):
        return (num+self.__smooth_alpha)/(total+dimension*self.__smooth_alpha)
    
class NumCounter(AttrCounter):
    def __init__(self, name):
        super().__init__(name)

    def prepare(self, class_num):
        self.__attr_data = [[] for i in range(class_num)]
        self.__mean = [0] * class_num
        self.__deviation = [0] * class_num
    
    def add(self, attr, class_index):
        self.__attr_data[class_index].append(float(attr))
    
    def run(self):
        for index, data in enumerate(self.__attr_data):
            mean = sum(data)/len(data)
            deviation = math.sqrt(sum([(x - mean)**2 for x in data])/(len(data)-1))
            self.__mean[index] = mean
            self.__deviation[index] = deviation
#         print(self.name, self.__mean, self.__deviation)
    
    def get(self, attr):
        return tuple(self._normal_distribution(float(attr), self.__mean[i], self.__deviation[i]) for i in range(len(self.__mean)))
    
    def _normal_distribution(self, val, mean, deviation):
        return 1/(deviation*math.sqrt(2*math.pi))*np.exp(-((val-mean)/deviation)**2/2) if deviation > 0 else 0
    
class Num2NomCounter(NomCounter):  
    def __init__(self, name, bins):
        assert(bins >= 2)
        super().__init__(name)
        self.__bins = bins

    def prepare(self, class_num):
        self.__split = [0] * (self.__bins-1)
        self.__attr_data = []
        self.__attr_count = [[0]*self.__bins for i in range(class_num)]
        self.__attr_prob = [[0]*self.__bins for i in range(class_num)]
    
    def add(self, attr, class_index):
        self.__attr_data.append((float(attr), class_index))
    
    def run(self):
        self.__attr_data.sort()
        gaps = self.__split_integer(len(self.__attr_data),self.__bins)
        cur = 0
        for i in range(self.__bins-1):
            npos = sum(gaps[:i+1])
            self.__split[i] = self.__attr_data[npos][0]
            for j in range(cur, npos):
                self.__attr_count[self.__attr_data[j][1]][i] += 1
            cur = npos
        for j in range(cur, len(self.__attr_data)):
            self.__attr_count[self.__attr_data[j][1]][i+1] += 1           
        for index,attrs in enumerate(self.__attr_count):
            total = sum(attrs)
            for val, num in enumerate(attrs):
                self.__attr_prob[index][val] = self._laplace_smooth(num, total, self.__bins)
        # print(self.__split)
        # print(self.__attr_count)
        # print(self.__attr_prob)
        # print("____________")
    
    def get(self, attr):
        attr = float(attr)
        for i in range(self.__bins):
            if i >= len(self.__split) or attr < self.__split[i]:
                break
        return tuple(prob[i] for prob in self.__attr_prob)
    
    def __split_integer(self, m, n):
        quotient = int(m / n)
        remainder = m % n
        if remainder > 0:
            return [quotient] * (n - remainder) + [quotient + 1] * remainder
        if remainder < 0:
            return [quotient - 1] * -remainder + [quotient] * (n + remainder)
        return [quotient] * n
    
class Nom2NumCounter(NumCounter):
    def __init__(self, name, mapper):
        super().__init__(name)
        self.__mapper = mapper
    
    def add(self, attr, class_index):
        self._attr_data[class_index].append(self.__mapper(attr))

class Model:
    def __init__(self, classes, *counters):
        self.__classes = {classname:index for index,classname in enumerate(classes)}
        self.__counters = counters
        self.__filters = ()
        self.__dump_firt_column = False
        for index, counter in enumerate(counters):
            if isinstance(counter, ClassCounter):
                self.__classpos = index
            counter.prepare(len(classes))
            
    def set_filters(self, filters):
        self.__filters = filters
        
    def set_dump_firt_column(self, enabled):
        self.__dump_firt_column = enabled
            
    def preprocess(self, lines):
        for line in lines:
            inputs = line.strip().split(",")[1:] if self.__dump_firt_column else line.strip().split(",")
            if not inputs or not inputs[0]:
                continue
            class_index = self.__classes[inputs[self.__classpos]]
            for i, val in enumerate(inputs):
                if val not in self.__filters:
                    if i == self.__classpos:
                        self.__counters[i].add(class_index)
                    else:
                        self.__counters[i].add(val, class_index)
                
    def train(self):
        for counter in self.__counters:
            counter.run()

    def test(self, lines):
        res = []
        for line in lines:
            inputs = line.strip().split(",")[1:] if self.__dump_firt_column else line.strip().split(",")
            if not inputs or not inputs[0]:
                continue
            real = self.__classes[inputs[self.__classpos]]
            predict = []
            for i, val in enumerate(inputs):
                if val not in self.__filters:
                    if i == self.__classpos:
                        predict.append(np.log2(np.array(self.__counters[i].get())))
                    else:
                        predict.append(np.log2(np.array(self.__counters[i].get(val))))
                        
                            
            res.append((real, np.argmax(reduce(lambda x, y: x+y, predict))))
        return res

def evaluate(result, classes):
    grid = np.zeros((len(classes), len(classes)),dtype=int)
    for instance in result:
        grid[instance[0]][instance[1]] += 1
    print(grid)



In [27]:
# filename = "datasets/adult.data"
# classes = ("<=50K" , ">50K")
# model = Model(
#     classes,
#     NumCounter("age"),
#     NomCounter("workclass"),
#     NumCounter("fnlwgt"),
#     NomCounter("education"),
#     NomCounter("education-num"),
#     NomCounter("marital-status"),
#     NomCounter("occupation"),
#     NomCounter("relationship"),
#     NomCounter("race"),
#     NomCounter("sex"),
#     NumCounter("capital-gain"),
#     NumCounter("capital-loss"),
#     NumCounter("hours-per-week"),
#     NomCounter("native-country"),
#     ClassCounter("salary")
# )
# model.set_filters(("?"))

# filename = "datasets/car.data"
# classes = ("unacc" , "acc", "good", "vgood")
# model = Model(
#     classes,
#     NomCounter("buying"),
#     NomCounter("maint"),
#     NomCounter("doors"),
#     NomCounter("persons"),
#     NomCounter("lug_boot"),
#     NomCounter("safety"),
#     ClassCounter("Car Evaluation")
# )

filename = "datasets/wine.data"
classes = ("1" , "2", "3")
model = Model(
    classes,
    ClassCounter("Alcohol"),
    NumCounter("age"),
    NumCounter("Malic acid"),
    NumCounter("Ash"),
    NumCounter("Alcalinity of ash"),
    NumCounter("Magnesium"),
    NumCounter("Total phenols"),
    NumCounter("Flavanoids"),
    NumCounter("Nonflavanoid phenols"),
    NumCounter("Proanthocyanins"),
    NumCounter("Color intensity"),
    NumCounter("Hue"),
    NumCounter("OD280/OD315 of diluted wines"),
    NumCounter("Proline")
)

with open(filename,"r") as file:
    lines = file.readlines()
    model.preprocess(lines)
    model.train()
    result = model.test(lines)
    evaluate(result, classes)

age [13.744745762711865, 12.278732394366198, 13.153749999999997] [0.4621253596612943, 0.5379642302964676, 0.5302413139918748]
Malic acid [2.0106779661016954, 1.932676056338028, 3.3337500000000007] [0.6885488598764588, 1.0155687467009085, 1.0879057081324806]
Ash [2.455593220338984, 2.244788732394365, 2.4370833333333333] [0.2271659820517139, 0.31546726864369473, 0.1846901756805375]
Alcalinity of ash [17.037288135593222, 20.238028169014086, 21.416666666666668] [2.5463224533110727, 3.3497704056582367, 2.2581609287519444]
Magnesium [106.33898305084746, 94.54929577464789, 99.3125] [10.498949322623579, 16.753497487641514, 10.890472607606213]
Total phenols [2.8401694915254234, 2.2588732394366198, 1.6787500000000002] [0.3389613523154669, 0.5453610843043734, 0.3569708552380176]
Flavanoids [2.982372881355932, 2.080845070422536, 0.7814583333333331] [0.3974936086367577, 0.7057007590815028, 0.293504065601863]
Nonflavanoid phenols [0.29, 0.363661971830986, 0.44749999999999995] [0.07004924376268554, 0